In [1]:
# Standard libraries
import sys
import math
import random
import time
import os

# Third-party imports
import numpy as np
import pandas as pd
import pickle as pkl

import tensorflow as tf

import importlib
from yapf.yapflib.yapf_api import FormatCode
import GPUtil

# Project imports
import ephesus
import utils

In [2]:
main_path = '/home/rfradkin/xom/'
raw_tess_path = '/scratch/data/tess/lcur/spoc/raws/'
tess_metad_path = '/data/scratch/data/tess/meta/'
xom_data_path = '/data/scratch/data/moon/'

### Sector Data Processing

In [3]:
forma_names = {
    'curve_injec': 'Curve Injected',
    'curve_type': 'Curve Type',
    'cut_times': 'Cut Times',
    'cut_trans': 'Cut Transit', 
    'cut_numbe': 'Cut Number',
    'cut_start_index': 'Cut Start Index',
    'detre_time_scale': 'Detrended Time Scale [days]',
    'detre_type': 'Detrended Type',
    'eb': 'Eclipsing Binary',
    'file_name': 'File Name',
    'injec_times': 'Injection Times',
    'inter_type': 'Interpolation Spline',
    'initi_paddi': 'Initial Padding',
    'linea_limb_darke_coeff': 'Linear Limb Darkening Coefficient',
    'max_ampli': 'Max Amplitude',
    'model_file': 'Model File',
    'moon_eccen': 'Moon Eccentricity',
    'moon_epoch': 'Moon Epoch [BJD]',
    'moon_incli': 'Moon Inclination [°]',
    'moon_mass': 'Moon Mass [M⨂]',
    'moon_numbe': 'Number of Moons',
    'moon_perio': 'Moon Period [days]',
    'moon_radiu': 'Moon Radius [R⨂]',
    'moon_sin_w': 'Planet Sin Argument of Periapsis [AU]',
    'moon_signa': 'Moon Signal',
    'plane_cut_injec': 'Cut Injected (Planet)',
    'plane_eccen': 'Planet Eccentricity',
    'plane_epoch': 'Planet Epoch [BJD]',
    'plane_incli': 'Planet Inclination [°]',
    'plane_mass': 'Planet Mass [M⨂]',
    'plane_moon_cut_injec': 'Cut Injected (Planet, Moon)',
    'plane_numbe': 'Number of Planets',
    'plane_perio': 'Planet Period [days]',
    'plane_radiu': 'Planet Radius [R⨂]',
    'plane_sin_w': 'Planet Sin Argument of Periapsis [AU]',
    'plane_signa': 'Planet Signal',
    'plane_type': 'Planet Type',
    'predi': 'Prediction',
    'quadr_limb_darke_coeff': 'Quadratic Limb Darkening Coefficient',
    'ratio_moon_plane_radiu': 'Ratio of Moon to Planetary Radius',
    'ratio_moon_stell_radiu': 'Ratio of Moon to Stellar Radius', 
    'ratio_plane_stell_radiu': 'Ratio of Planetary to Stellar Radius',
    'signa': 'Signal',
    'stell_effec_tempe': 'Stellar Effective Temperature [C°]',
    'stell_magni': 'Stellar Magnitude',
    'stell_mass': 'Stellar Mass [M☉]',
    'stell_radiu': 'Stellar Radius [R☉]',
    'tic_id': 'TIC ID',
    'toi': 'TOI',
    'trape_trans': 'Trapezoidal Transits',
    'type_calcu': 'Type Calculation',
    'type_limb_darke': 'Type of Limb Darkening',
    'type_orbit_archi': 'Type of Orbital Architecture'
}

infor = {
    'curve_injec': False, 
    'curve_type': None,
    'cut_times': None,
    'cut_trans': None,
    'cut_numbe': None,
    'cut_start_index': None,
    'detre_time_scale': None,
    'detre_type': None,
    'eb': None,
    'file_name': None,
    'forma_names': forma_names,
    'injec_times': None,
    'inter_type': None,
    'initi_paddi': None,
    'linea_limb_darke_coeff': None,
    'max_ampli': None,
    'model_file': None,
    'moon_eccen': None,
    'moon_epoch': None,
    'moon_incli': None,
    'moon_mass': None,
    'moon_numbe': None,
    'moon_perio': None,
    'moon_radiu': None,
    'moon_sin_w': None,
    'moon_signa': None,
    'plane_cut_injec': None,
    'plane_eccen': None,
    'plane_epoch': None,
    'plane_incli': None,
    'plane_mass': None,
    'plane_moon_cut_injec': None,
    'plane_numbe': None,
    'plane_perio': None,
    'plane_radiu': None,
    'plane_sin_w': None,
    'plane_signa': None,
    'plane_type': None,
    'predi': None,
    'quadr_limb_darke_coeff': None,
    'ratio_moon_plane_radiu': None,
    'ratio_moon_stell_radiu': None,
    'ratio_plane_stell_radiu': None,
    'signa': None,
    'stell_effec_tempe': None,
    'stell_magni': None,
    'stell_mass': None,
    'stell_radiu': None,
    'tic_id': None,
    'toi': None,
    'trape_trans': None,
    'type_calcu': None,
    'type_limb_darke': None,
    'type_orbit_archi': None
}

In [4]:
%%capture

numbe_of_curve = 0
TIC8 = pd.read_csv(f'{tess_metad_path}nomi_miss_2min_info.csv')
# detre_scale = 0.4
# detre_type = 'spln'
stand_lengt = 1901
min_lengt = 600
max_time_gap = 10  # mins
caden = 2  # mins
spln_type = 'cubic'

for curre_secto in range(1, 27):
    from_secto = curre_secto
    to_secto = from_secto + 1
    
    files = utils.retur_secto_files([from_secto, to_secto], f'{raw_tess_path}')
    compl_set = ''
    if numbe_of_curve: 
        sampl = utils.retur_rando_sampl(numbe_of_curve, len(files))
        files = [files[index] for index in sampl]
        compl_set = '-parti'
    
    light_curve = utils.retur_fits_data(files, infor, f'{raw_tess_path}')
    # Find TIC ID of all TOIs
    toi_full = pd.read_csv(f'{tess_metad_path}toi.csv')
    toi_id = toi_full['TIC ID'].to_numpy()
    # Mark TOIs in light curve dataset
    utils.mark_TOI(light_curve, toi_id)

    for i in range(len(light_curve)):
        TIC_ID = light_curve[i, -1, 1]['tic_id']
        # Find current TIC ID in TIC8
        star_infor = TIC8[TIC8.tici == TIC_ID]
        # Make sure TIC ID is in TIC8
        if len(star_infor):
            # Gather stellar radius, mass, temperature, and magnitude
            if str(star_infor.radistar.iloc[0]) != 'nan':
                light_curve[i, -1, 1]['stell_radiu'] = star_infor.radistar.iloc[0]
            if str(star_infor.massstar.iloc[0]) != 'nan':
                light_curve[i, -1, 1]['stell_mass'] = star_infor.massstar.iloc[0]
            if str(star_infor.tmptstar.iloc[0]) != 'nan':
                light_curve[i, -1, 1]['stell_effec_tempe'] = \
    star_infor.tmptstar.iloc[0]
            if str(star_infor.tmag.iloc[0]) != 'nan':
                light_curve[i, -1, 1]['stell_magni'] = star_infor.tmag.iloc[0]

    # Save data
    with open(f'{xom_data_path}raw_data/\
forma_curve-{from_secto}_{to_secto}{compl_set}.pkl','wb') as f:
        pkl.dump(light_curve, f)
 
#     detre_curve = np.copy(light_curve)
#     # Copy dictionaries
#     for i in range(len(detre_curve)):
#         detre_curve[i, -1, 1] = detre_curve[i, -1, 1].copy()

#     for i in range(len(detre_curve)):
#         curre_detre = ephesus.bdtr_tser(
#             time=detre_curve[i, utils.find_start(detre_curve[i]):-1,
#                              0].astype(float),
#             lcur=detre_curve[i, utils.find_start(detre_curve[i]):-1,
#                              1].astype(float),
#             typebdtr=detre_type,
#             timescalbdtrspln=detre_scale,
#         )[0]
#         full_detre = curre_detre[0]
#         # Take detrended chunks and turn them into one curve
#         for j in range(1, len(curre_detre)):
#             full_detre = np.concatenate((full_detre, curre_detre[j]))

#         detre_curve[i, utils.find_start(detre_curve[i]):-1, 1] = full_detre
#         # Set feature values
#         detre_curve[i, -1, 1]['curve_type'] = 'Detrended Curve'
#         detre_curve[i, -1, 1]['detre_time_scale'] = detre_scale
#         detre_curve[i, -1, 1]['detre_type'] = detre_type
        
    list_cuts = []
    for i in range(len(light_curve)):
        pre_cut = np.copy(light_curve[i])
        pre_cut[-1, 1] = pre_cut[-1, 1].copy()
        # Insert 'inter_spot' for later interpolation
        pre_cut = utils.inser_inter_spot(pre_cut, max_time_gap, caden)
        # Cut curve and assign cut times to the detrended curve
        pre_cut = utils.cut_curve(pre_cut, max_time_gap, min_lengt, stand_lengt)
        # Cut number
        cut_numbe = 0
        # Find used cut times
        cut_times = []
        for curre_cut in pre_cut:
            # Remove all cuts of only [0,0]
            if curre_cut[-2, 0]: 
                # Interpolate each cut
                cut = utils.inter_curve(curre_cut, spln_type)
                # Assign feature values
                if cut[-1, 1]['type_orbit_archi'] == 'planmoon':
                    cut[-1, 1]['plane_moon_cut_injec'] = utils.retur_curve_injec_statu(cut)
                    cut[-1, 1]['plane_cut_injec'] = False
                elif cut[-1, 1]['type_orbit_archi'] == 'plan':
                    cut[-1, 1]['plane_moon_cut_injec'] = False
                    cut[-1, 1]['plane_cut_injec'] = utils.retur_curve_injec_statu(cut)
                else:
                    cut[-1, 1]['plane_moon_cut_injec'] = False
                    cut[-1, 1]['plane_cut_injec'] = False
                cut[-1, 1]['cut_trans'] = cut[-1, 1]['plane_moon_cut_injec'] or cut[-1, 1]['plane_cut_injec']
                cut[-1, 1]['cut_numbe'] = cut_numbe
                cut[-1, 1]['inter_type'] = spln_type
                # Assign cut to list containing the rest of the cuts
                list_cuts.append(cut.tolist())
                # Increase cut number
                cut_numbe += 1
                cut_times.append([cut[utils.find_start(cut), 0], cut[-2, 0]])
        # Assign cut times to the corresponding light and detrended curve
        light_curve[i, -1, 1]['cut_times'] = cut_times
#         detre_curve[i, -1, 1]['cut_times'] = cut_times

    padde_cuts = tf.keras.preprocessing.sequence.pad_sequences(list_cuts,
                                                               maxlen=stand_lengt,
                                                               dtype=object)
        
    # Save data
    detre_status = 'detre-' if padde_cuts[0, -1,
                                         1]['detre_time_scale'] else 'not_detre-'
    padde_cuts_path = f'{xom_data_path}padde_cuts/padde_cuts-{detre_status}{from_secto}_{to_secto}{compl_set}.pkl'
    with open(padde_cuts_path, 'wb') as f:
        pkl.dump(padde_cuts, f)

In [5]:
utils.send_task_comple_email('redo raw cuts')

### Formatting